In [11]:
!pip3 install lxml bs4

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 115 kB 2.0 MB/s 
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=46c52a9dc6721be7fd97521ad737c5543c398407cbe99626febcbe4177599ded
  Stored in directory: /home/conradbez/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [17]:
import pandas as pd
import re
import requests
import bs4 as bs

In [18]:
points_table = pd.DataFrame()
for round in range(1,18):
  round_points_table  =pd.read_html(f'https://www.footywire.com/afl/footy/dream_team_round?year=2020&round={round}&p=&s=T')[9]

  round_points_table['Round'] = f'Round {round}'
  points_table = points_table.append(round_points_table)

points_table.columns = points_table.iloc[0]
points_table = points_table.rename({'Round 1':'Round'},axis=1)
points_table = points_table[['Player', 'Team',"2020 R1Score",'Round']]
points_table

,Player,Team,2020 R1Score,Round
0,Player,Team,2020 R1Score,Round 1
1,Jack Viney Injured,Demons,132,Round 1
2,Dylan Shiel,Bombers,122,Round 1
3,Ben Cunnington,Kangaroos,117,Round 1
4,Luke Shuey Injured,Eagles,116,Round 1
...,...,...,...,...
392,Brandon Zerk-Thatcher,Bombers,12,Round 17
393,Sam Docherty,Blues,11,Round 17
394,Toby McLean Injured,Bulldogs,10,Round 17
395,Lewis Taylor,Swans,0,Round 17


In [19]:
replace_team_names =   {"Adelaide":"Crows",   "Brisbane":"Lions","Carlton":"Blues",   "Collingwood":"Magpies",   "Essendon":"Bombers",   "Fremantle":"Dockers",   "GWS":"Giants",   "Geelong":"Cats",   "Gold Coast":"Suns",   "Hawthorn":"Hawks",   "Melbourne":"Demons",   "North Melbourne":"Kangaroos",   "Port Adelaide":"Power",   "Richmond":"Tigers",   "St Kilda":"Saints",   "Sydney":"Swans",   "West Coast":"Eagles",   "Western Bulldogs":"Bulldogs"}

In [23]:
def getRounds(r=2020):
  rounds = pd.read_html(f'https://www.footywire.com/afl/footy/ft_match_list?year={r}')[8]
  rounds['Round_Num'] = rounds [0].str.extract(r'(Round \d{1,2})')
  rounds['Round_Num'] = rounds['Round_Num'].ffill()
  rounds['Team 1'] = rounds[1].str.split(' v ',expand=True)[0]
  rounds['Team 2'] = rounds[1].str.split(' v ',expand=True)[1]
  rounds = rounds.dropna(subset=['Team 2'])
  rounds.columns = rounds.iloc[0]
  rounds = rounds[rounds['Home'] != 'Home']
  rounds = rounds[['Round 1',	'Home',	'Away Teams', 'Date']]
  rounds = rounds.replace(replace_team_names)
  home_teams = rounds['Home'].unique()
  for t in points_table['Team']:
    if t not in home_teams:
      print(t)
      print('are the team mismatches')
  rounds = rounds.append(rounds.rename({'Home': 'Away Teams', 'Away Teams':'Home'},axis=1))
  rounds = rounds.reset_index(drop=True).rename({'Home': 'Team 1', 'Away Teams':'Team 2'},axis=1)
  rounds = rounds.rename({'Round 1': 'Round'},axis=1)
  rounds['Round'] = rounds['Round'].str.slice(start=6)
  return rounds
rounds = getRounds(2020)
nextRounds = getRounds(2021)
nextRounds.to_csv('2021_schedule')

Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches
Team
are the team mismatches


In [ ]:
Players = points_table.merge(rounds, left_on=['Team', 'Round'], right_on=['Team 1', 'Round'])

In [ ]:
Players = pd.DataFrame()
position = 'RU'
for position in ['RU','DE','FO','MI']:
  f = requests.get(f"https://www.footywire.com/afl/footy/dream_team_season?p={position}").text
  soup = bs.BeautifulSoup(f, 'lxml')
  table = soup.find_all('table')[8] 
  parsed_table = soup.find_all('table')[8] 
  data = [[td.a['href'] if td.find('a') else 
            ''.join(td.stripped_strings)
            for td in row.find_all('td')]
          for row in parsed_table.find_all('tr')]
  data2 = [[''.join(td.stripped_strings)
            for td in row.find_all('td')]
          for row in parsed_table.find_all('tr')]
  df = pd.DataFrame(data[1:], columns=data[0])
  df2 = pd.DataFrame(data2[1:], columns=data2[0])
  player_info = df2.join(df['Player'],rsuffix='_url')
  player_info['Position'] = position
  Players = Players.append(player_info)
Players

,Rank,Player,Team,Games,Price,TotalScore,AverageScore,*Value,Player_url,Position
0,1,Max Gawn,Demons,14,"$944,000","1,385",98.9,10.5,pr-melbourne-demons--max-gawn,RU
1,2,Brodie Grundy,Magpies,17,"$867,000","1,546",90.9,10.5,pr-collingwood-magpies--brodie-grundy,RU
2,3,Reilly O'Brien,Crows,17,"$828,000","1,475",86.8,10.5,pr-adelaide-crows--reilly-o-brien,RU
3,4,Todd Goldstein,Kangaroos,17,"$760,000","1,353",79.6,10.5,pr-kangaroos--todd-goldstein,RU
4,5,Rowan Marshall,Saints,17,"$738,000","1,314",77.3,10.5,pr-st-kilda-saints--rowan-marshall,RU
...,...,...,...,...,...,...,...,...,...,...
204,204,Deven Robertson,Lions,1,"$186,000",26,26.0,14.0,pr-brisbane-lions--deven-robertson,MI
205,206,Josh Honey,Blues,1,"$170,000",22,22.0,12.9,pr-carlton-blues--josh-honey,MI
206,206,Hamish Brayshaw,Eagles,1,"$172,000",22,22.0,12.8,pr-west-coast-eagles--hamish-brayshaw,MI
207,206,Timothy Broomhead,Magpies,1,"$298,000",22,22.0,7.4,pr-collingwood-magpies--timothy-broomhead,MI


In [ ]:
def getPlayerScores(pu):
  pu = "https://www.footywire.com/afl/footy/"+pu
  fantasy_points = pd.read_html(pu)
  playerTable = pd.DataFrame()
  for n,g in enumerate(fantasy_points):
    if "AFL Fantasy Stats" in str(g.iloc[0,0]):
      tempplayerTable = fantasy_points[n+1]
      tempplayerTable['Year'] = g.iloc[0,0]
      playerTable = playerTable.append(tempplayerTable)
  playerTable = playerTable.rename({0:"Round",2:'Score'},axis=1)[['Round','Year','Score']]
  playerTable = playerTable[playerTable['Round'].str.len()<=2]
  playerTable['Year'] = playerTable['Year'].str.slice(stop=4)
  return playerTable

In [ ]:
full_player_scores = pd.DataFrame()
for index, row in Players.iterrows():
  playerScore = getPlayerScores(row['Player_url'])
  playerScore['Name'] = row['Player']
  playerScore['Team'] = row['Team']
  playerScore['Position'] = row['Position']
  playerScore['Price'] = int(row['Price'].replace(',', '').replace('$', ''))
  full_player_scores = full_player_scores.append(rounds.merge(playerScore,left_on=['Team 1','Round'], right_on=['Team', 'Round']))
full_player_scores
full_player_scores.to_csv('full_players.csv')

NameError: ignored